# Pandas (continues)

In [ ]:
import pandas as pd
import numpy as np

## Creation of dataframes

The DataFrame is essentially a two dimensional object, and it can be created in three different ways:

* out of a two dimensional NumPy array
* out of given columns
* out of given rows

### Creating DataFrames from a NumPy array

In the following example a DataFrame with 2 rows and 3 column is created. The row and column indices are given explicitly.

In [ ]:
df=pd.DataFrame(np.random.randn(2,3), columns=["First", "Second", "Third"], index=["a", "b"])
df

Note that now both the rows and columns can be accessed using the special `Index` object:

In [ ]:
df.index                            # These are the "row names"

In [ ]:
df.columns                          # These are the "column names"

If either `columns` or `index` argument is left out, then an implicit integer index will be used:

In [ ]:
df2=pd.DataFrame(np.random.randn(2,3), index=["a", "b"])
df2

Now the column index is an object similar to Python's builtin `range` type:

In [ ]:
df2.columns

### Creating DataFrames from columns

A column can be specified as a list, an NumPy array, or a Pandas' Series. The names of the columns can be given either with the `columns` parameter, or if Series objects are used, then the `name` attribute of each Series is used as the column name.

In [ ]:
s1 = pd.Series([1,2,3])
s1

In [ ]:
s2 = pd.Series([4,5,6], name="b")
s2

Give the column name explicitly:

In [ ]:
pd.DataFrame(s1, columns=["a"])

Use the `name` attribute of Series s2 as the column name:

In [ ]:
pd.DataFrame(s2)

If using multiple columns, then they must be given as the dictionary, whose keys give the column names and values are the actual column content.

In [ ]:
pd.DataFrame({"a": s1, "b": s2})

### Creating DataFrames from rows

We can give a list of rows as a parameter to the DataFrame constructor. Each row is given as a dict, list, Series, or NumPy array. If we want to give names for the columns, then either the rows must be dictionaries, where the key is the column name and the values are the elements of the DataFrame on that row and column, or else the column names must be given explicitly. An example of this:

In [ ]:
df=pd.DataFrame([{"Wage" : 1000, "Name" : "Jack", "Age" : 21, }, {"Wage" : 1500, "Name" : "John", "Age" : 29}])
df

Or:

In [ ]:
df = pd.DataFrame([[1000, "Jack", 21], [1500, "John", 29]], columns=["Wage", "Name", "Age"])
df

Or:

In [ ]:
df = pd.DataFrame({"Wage" : [1000, 1500], 
                   "Name" : ["Jack", "John"], 
                   "Age" : [21, 29]})
df

## Accessing columns and rows of a dataframe

Even though DataFrames are basically just two dimensional arrays, the way to access their elements is different from NumPy arrays. There are a couple of complications, which we will go through in this section.

Firstly, the bracket notation `[]` does not allow the use of an index pair to access a single element of the DataFrame. Instead only one dimension can be specified.

Well, does this dimension specify the rows of the DataFrame, like NumPy arrays if only one index is given, or does it specify the columns of the DataFrame?

It depends!

If an integer is used, then it specifies a column of the DataFrame in the case the **explicit** indices for the column contain that integer. In any other case an error will result. For example, with the above DataFrame, the following indexing will not work, because the explicit column index consist of the column names "Name" and "Wage" which are not integers.

In [ ]:
try:
    df[0]
except KeyError:
    import sys
    print("Key error", file=sys.stderr)

The following will however work.

In [ ]:
df["Wage"]

In [ ]:
df.Wage

As does the fancy indexing:

In [ ]:
df[["Wage", "Name"]]

If one indexes with a slice or a boolean mask, then the **rows** are referred to. Examples of these:

In [ ]:
df[0:1]                           # slice

In [ ]:
df[df.Wage > 1200]               # boolean mask

If some of the above calls return a Series object, then you can chain the bracket calls to get a single value from the DataFrame:

In [ ]:
df["Wage"][1]                    # Note order of dimensions

But there is a better way to achieve this, which we will see in the next section.

## Alternative indexing and data selection

If the explanation in the previous section sounded confusing or ambiguous, or if you didn't understand a thing, you don't have to worry.

There is another way to index Pandas DataFrames, which

* allows use of index pairs to access a single element
* has the same order of dimensions as NumPy: first index specifies rows, second columns
* is not ambiguous about implicit or explicit indices

Pandas DataFrames have attributes `loc` and `iloc` that have the above qualities.
You can use `loc` and `iloc` attributes and forget everything about the previous section. Or you can use these attributes
and sometimes use the methods from the previous section as shortcuts if you understand them well.

The difference between `loc` and `iloc` attributes is that the former uses explicit indices and the latter uses the implicit integer indices. Examples of use:

In [ ]:
df

In [ ]:
df.loc[1, "Wage"]

In [ ]:
df.iloc[-1,-1]             # Right lower corner of the DataFrame

In [ ]:
df.loc[1, ["Name", "Wage"]]

With `iloc` everything works like with NumPy arrays: indexing, slicing, fancy indexing, masking and their combinations. With `loc` it is the same but now the names in the explicit indices are used for specifying rows and columns. Make sure your understand why the above examples work as they do!

## Summary statistics

The summary statistic methods work in a similar way as their counter parts in NumPy. By default, the aggregation is done over columns.

In [ ]:
wh = pd.read_csv("https://www.cs.helsinki.fi/u/jttoivon/dap/data/fmi/kumpula-weather-2017.csv")

In [ ]:
wh2 = wh.drop(["Year", "m", "d"], axis=1)  # taking averages over these is not very interesting
wh2.mean()

The `describe` method of the `DataFrame` object gives different summary statistics for each (numeric) column. The result is a DataFrame. This method gives a good overview of the data, and is typically used in the exploratory data analysis phase.

In [ ]:
wh.describe()

## Missing data

- You may have noticed something strange in the output of the `describe` method. 
- The minimum value in both precipitation and snow depth fields is -1. The special value -1 means that on that day there was absolutely no snow or rain, whereas the value 0 might indicate that the value was close to zero. Secondly, the snow depth column has count 358, whereas the other columns have count 365, one measurement/value for each day of the year. How is this possible? Every field in a DataFrame should have the same number of rows. Let's use the `unique` method of the Series object to find out, which different values are used in this column:

In [ ]:
wh["Snow depth (cm)"].unique()

The `float` type allows a special value `nan` (Not A Number), in addition to normal floating point numbers. This value can represent the result from an illegal operation. For example, the operation 0/0 can either cause an exception to occur or just silently produce a `nan`. In Pandas `nan` can be used to represent a missing value. In the weather DataFrame the `nan` value tells us that the measurement from that day is not available, possibly due to a broken measuring instrument or some other problem.

Note that only float types allow the `nan` value (in Python, NumPy or Pandas). So, if we try to create an integer series with missing values, its dtype gets promoted to `float`:

In [ ]:
pd.Series([1,3,2])

In [ ]:
pd.Series([1,3,2, np.nan])

For non-numeric types the special value `None` is used to denote a missing value, and the dtype is promoted to `object`.

In [ ]:
pd.Series(["jack", "joe", None]).isnull()

Pandas excludes the missing values from the summary statistics, like we saw in the previous section. Pandas also provides some functions to handle missing values.

The missing values can be located with the `isnull` method:

In [ ]:
wh.isnull()      # returns a boolean mask DataFrame

This is not very useful as we cannot directly use the mask to index the DataFrame. We can, however, combine it with the `any` method to find out all the rows that contain at least one missing value:

In [ ]:
wh[wh.isnull().any(axis=1)]

The `notnull` method works conversively to the `isnull` method.

The `dropna` method of a DataFrame drops columns or rows that contain missing values from the DataFrame, depending on the `axis` parameter.

In [ ]:
wh.dropna().shape   # Default axis is 0

In [ ]:
wh.dropna(axis=1).shape # Drops the columns containing missing values

The `how` and `thresh` parameters of the `dropna` method allow one to specify how many values need to be missing in order for the row/column to be dropped.

The `fillna` method allows to fill the missing values with some constant or interpolated values. The `method` parameter can be:

* `None`: use the given positional parameter as the constant to fill missing values with
* `ffill`: use the previous value to fill the current value
* `bfill`: use the next value to fill the current value

For example, for the weather data we could use forward fill

In [ ]:
wh = wh.fillna(method='ffill')
wh[wh.isnull().any(axis=1)]

## Converting columns from one type to another

There are several ways of converting a column to another type. For converting single columns (a Series) one can use the `pd.to_numeric` function or the `map` method. For converting several columns in one go one can use the `astype` method. We will give a few examples of use of these methods/functions. For more details, look from the Pandas documentation.

In [ ]:
pd.Series(["1","2"]).map(int)                           # str -> int

In [ ]:
pd.Series([1,2]).map(str)                               # int -> str

In [ ]:
pd.to_numeric(pd.Series([1,1.0]), downcast="integer")   # object -> int

In [ ]:
pd.to_numeric(pd.Series([1,"a"]), errors="coerce")      # conversion error produces Nan

In [ ]:
pd.Series([1,2]).astype(str)                            # works for a single series

In [ ]:
df = pd.DataFrame({"a": [1,2,3], "b" : [4,5,6], "c" : [7,8,9]})
print(df.dtypes)
print(df)

In [ ]:
df.astype(float)                       # Convert all columns

In [ ]:
df2 = df.astype({"b" : float, "c" : str})    # different types for columns
print(df2.dtypes)
print(df2)

## String processing

If the elements in a column are strings, then the vectorized versions of Python's string processing methods are available. These are accessed through the `str` attribute of a Series or a DataFrame. For example, to capitalize all the strings of a Series, we can use the `str.capitalize` method:

In [ ]:
names = pd.Series(["joseph", "boris", "angela", "vladimir"])
names.str.capitalize()

One can find all the available methods by pressing the tab key after the text `names.str.` in a Python prompt. Try it in below cell!

In [ ]:
names.str.upper()

We can split a column or Series into several columns using the `split` method. For example:

In [ ]:
full_names = pd.Series(["Joseph Biden", "Boris Johnson", "Angela Merkel", "Vladimir Putin"])
full_names.str.split()

This is not exactly what we wanted: now each element is a list. We need to use the `expand` parameter to split into columns:

In [ ]:
full_names.str.split(expand=True)